In [1]:
import pickle
import numpy as np
import tensorflow as tf

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [3]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks):
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1))
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
        test = unpickle(pos + '/test_batch')
        Xte = test[b'data']
        Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

In [4]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, 10))
    Yte_onehot = np.zeros((Yte.size, 10))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot

In [5]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)
                                 
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")

# label data of train and test data, label data is represented by one-hot encoding
Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)


In [6]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

In [7]:
epsilon = 1e-3
### Implement the layers of CNNs 
W1 = tf.Variable(tf.random_normal([3,3,3,32], stddev=0.01))
# padding = 'SAME' keeps output size equal to input.

L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
# Pooling layer
#

In [8]:
batch_mean1, batch_var1 = tf.nn.moments(L1,[0])
scale1 = tf.Variable(tf.ones([32, 32, 32]))
beta1 = tf.Variable(tf.zeros([32, 32, 32]))
BN1 = tf.nn.batch_normalization(L1,batch_mean1,batch_var1,beta1,scale1,epsilon)
L1 = tf.nn.relu(BN1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
batch_mean2, batch_var2 = tf.nn.moments(L2,[0])
scale2 = tf.Variable(tf.ones([16, 16, 64]))
beta2 = tf.Variable(tf.zeros([16, 16, 64]))
BN2 = tf.nn.batch_normalization(L2,batch_mean2,batch_var2,beta2,scale2,epsilon)
L2 = tf.nn.relu(BN2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [10]:
W3 = tf.Variable(tf.random_normal([8*8*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 8*8*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
# dropout
L3 = tf.nn.dropout(L3, dropout_prob)

In [11]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [17]:
# cost function, you can change the implementation
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [18]:
# initialize the variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [19]:
### Implement the train process ###
batch_size = 100
total_batch = int(50000 / batch_size)

In [20]:
### Implement the test process ###

In [21]:
#~10
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs = Xtr[batch_size*i:batch_size*(i+1)]
        batch_ys = Ytr_onehot[batch_size*i:batch_size*(i+1)]

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          dropout_prob: 0.4})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1),
         'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
    print('Accuracy', sess.run(accuracy, 
                               feed_dict={
                                   X: Xte,
                                   Y: Yte_onehot,
                                   dropout_prob: 1}))

Epoch: 0001 Avg. cost =  1.722
Accuracy 0.5165
Epoch: 0002 Avg. cost =  1.355
Accuracy 0.5926
Epoch: 0003 Avg. cost =  1.203
Accuracy 0.6249
Epoch: 0004 Avg. cost =  1.106
Accuracy 0.6554
Epoch: 0005 Avg. cost =  1.033
Accuracy 0.6747
Epoch: 0006 Avg. cost =  0.982
Accuracy 0.6857
Epoch: 0007 Avg. cost =  0.943
Accuracy 0.6952
Epoch: 0008 Avg. cost =  0.909
Accuracy 0.7086
Epoch: 0009 Avg. cost =  0.877
Accuracy 0.7114
Epoch: 0010 Avg. cost =  0.853
Accuracy 0.7159


In [22]:
#~20
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs = Xtr[batch_size*i:batch_size*(i+1)]
        batch_ys = Ytr_onehot[batch_size*i:batch_size*(i+1)]

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          dropout_prob: 0.4})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1),
         'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
    print('Accuracy', sess.run(accuracy, 
                               feed_dict={
                                   X: Xte,
                                   Y: Yte_onehot,
                                   dropout_prob: 1}))

Epoch: 0001 Avg. cost =  0.825
Accuracy 0.7204
Epoch: 0002 Avg. cost =  0.800
Accuracy 0.7302
Epoch: 0003 Avg. cost =  0.775
Accuracy 0.7307
Epoch: 0004 Avg. cost =  0.762
Accuracy 0.732
Epoch: 0005 Avg. cost =  0.738
Accuracy 0.738
Epoch: 0006 Avg. cost =  0.720
Accuracy 0.7365
Epoch: 0007 Avg. cost =  0.700
Accuracy 0.7393
Epoch: 0008 Avg. cost =  0.683
Accuracy 0.7414
Epoch: 0009 Avg. cost =  0.668
Accuracy 0.7464
Epoch: 0010 Avg. cost =  0.652
Accuracy 0.7476


In [23]:
#~30
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs = Xtr[batch_size*i:batch_size*(i+1)]
        batch_ys = Ytr_onehot[batch_size*i:batch_size*(i+1)]

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          dropout_prob: 0.4})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1),
         'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
    print('Accuracy', sess.run(accuracy, 
                               feed_dict={
                                   X: Xte,
                                   Y: Yte_onehot,
                                   dropout_prob: 1}))

Epoch: 0001 Avg. cost =  0.632
Accuracy 0.7494
Epoch: 0002 Avg. cost =  0.619
Accuracy 0.7505
Epoch: 0003 Avg. cost =  0.605
Accuracy 0.7514
Epoch: 0004 Avg. cost =  0.592
Accuracy 0.7526
Epoch: 0005 Avg. cost =  0.582
Accuracy 0.7552
Epoch: 0006 Avg. cost =  0.565
Accuracy 0.7509
Epoch: 0007 Avg. cost =  0.551
Accuracy 0.7538
Epoch: 0008 Avg. cost =  0.537
Accuracy 0.7591
Epoch: 0009 Avg. cost =  0.521
Accuracy 0.7596
Epoch: 0010 Avg. cost =  0.514
Accuracy 0.7579


In [24]:
#~35
for epoch in range(5):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs = Xtr[batch_size*i:batch_size*(i+1)]
        batch_ys = Ytr_onehot[batch_size*i:batch_size*(i+1)]

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          dropout_prob: 0.4})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1),
         'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
    print('Accuracy', sess.run(accuracy, 
                               feed_dict={
                                   X: Xte,
                                   Y: Yte_onehot,
                                   dropout_prob: 1}))

Epoch: 0001 Avg. cost =  0.498
Accuracy 0.7633
Epoch: 0002 Avg. cost =  0.490
Accuracy 0.7626
Epoch: 0003 Avg. cost =  0.475
Accuracy 0.7627
Epoch: 0004 Avg. cost =  0.468
Accuracy 0.7617
Epoch: 0005 Avg. cost =  0.455
Accuracy 0.759


In [25]:
for epoch in range(5):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs = Xtr[batch_size*i:batch_size*(i+1)]
        batch_ys = Ytr_onehot[batch_size*i:batch_size*(i+1)]

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          dropout_prob: 0.4})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1),
         'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
    print('Accuracy', sess.run(accuracy, 
                               feed_dict={
                                   X: Xte,
                                   Y: Yte_onehot,
                                   dropout_prob: 1}))

Epoch: 0001 Avg. cost =  0.447
Accuracy 0.7633
Epoch: 0002 Avg. cost =  0.435
Accuracy 0.7666


KeyboardInterrupt: 